In [ ]:
pip install qutip

In [ ]:
from qutip import *
import numpy as np
import math
from scipy.stats import unitary_group
import matplotlib.pyplot as plt
import matplotlib
import numpy.linalg as la
import itertools
from tqdm import tqdm
from scipy.linalg import block_diag
from scipy.optimize import root
from scipy.optimize import minimize
from IPython.display import display
from tqdm.notebook import tnrange
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def ham_energies(N):
    energies = []
    for i in range(N+1):
        energies+=([i]*math.comb(N, i))
    return energies

def free_ham(N):
    ham0 = fock_dm(2,1)
    free_ham = 0
    for i in range(N):
        ham = [qeye(2)]*N
        ham[i] = ham0
        free_ham += tensor(ham)
    
    return free_ham

def int_ham(N):
    int_ham = 0
    for i in range(N):
        ham = [qeye(2)]*N
        if i==N-1:
            ham[i] = sigmax()
            ham[0] = sigmax()
        else:
            ham[i] = sigmax()
            ham[i+1] = sigmax()

        int_ham += tensor(ham)
    
    return int_ham

def spin_ham(N, J):
    return free_ham(N)+J*int_ham(N)

def min_energy(state):
    N = len(state.dims[0])
    eigenvalues, eigenstates = state.eigenstates()
    sorted_eigen = sorted(eigenvalues, reverse=True)
    energies = ham_energies(N)
        
    return np.dot(sorted_eigen, energies)

def ergotropy(state):
    N = len(state.dims[0])
    return (state*free_ham(N)).tr()-min_energy(state)

def energy(state):
    N = len(state.dims[0])
    return (state*free_ham(N)).tr()

def free_energy(state,T):
    N = len(state.dims[0])
    return (state*free_ham(N)).tr()- T*entropy_vn(state)

def calc_mean_ergotropy(state, N, k):
    numbers = [i for i in range(N)]
    combinations = list(itertools.combinations(numbers, k))

    erg = 0
    for comb in combinations:
        marginal_state = state.ptrace(comb)
        erg += ergotropy(marginal_state)
    
    return erg/len(combinations)

def dist_ergotropy(state, N, k):
    numbers = [i for i in range(N)]
    combinations = list(itertools.combinations(numbers, k))

    erg = []
    for comb in combinations:
        marginal_state = state.ptrace(comb)
        erg.append(ergotropy(marginal_state))
    
    return erg

def random_vector(dim):
    vector = []
    norm = 0 
    for i in range(dim):
        c = np.random.rand() + 1j * np.random.rand()
        norm += abs(c)**2
        vector.append(c)
    return np.array(vector)/np.sqrt(norm)

def random_product_state(N):
    state = []
    for i in range(N):
        state.append(Qobj(random_vector(2)))
        
    return(tensor(state))

def random_total_state(N):
    state = Qobj(random_vector(2**N))
    state.dims = [[2]*N, [1]*N]
    return state

def calc_temp(state):
    p1 = state.diag()[1]
    p0 = state.diag()[0]
    
    return 1/np.log(p0/p1)

def calc_mean_temp(state, N):
    
    sum_temp = 0
    for i in range(N):
        partial_state = state.ptrace(i)
        sum_temp += calc_temp(partial_state)
        
    return sum_temp/N

def calc_mean_free_energy(state, T, N, k):
    numbers = [i for i in range(N)]
    combinations = list(itertools.combinations(numbers, k))

    sum_free_energy = 0
    for comb in combinations:
        marginal_state = state.ptrace(comb)
        sum_free_energy += free_energy(marginal_state,T)
    
    return sum_free_energy/len(combinations)

def calc_mean_min_energy(state, N, k):
    numbers = [i for i in range(N)]
    combinations = list(itertools.combinations(numbers, k))

    sum_min_energy = 0
    for comb in combinations:
        marginal_state = state.ptrace(comb)
        sum_min_energy += min_energy(marginal_state)
        
    return sum_min_energy/len(combinations)

def calc_mean_energy(state, N, k):
    numbers = [i for i in range(N)]
    combinations = list(itertools.combinations(numbers, k))

    sum_energy = 0
    for comb in combinations:
        marginal_state = state.ptrace(comb)
        sum_energy += energy(marginal_state)
    
    return sum_energy/len(combinations)

def calc_mean_entropy(state, N, k):
    numbers = [i for i in range(N)]
    combinations = list(itertools.combinations(numbers, k))

    sum_entropy = 0
    for comb in combinations:
        marginal_state = state.ptrace(comb)
        sum_entropy += entropy_vn(marginal_state)
    
    return sum_entropy/len(combinations)

def calc_partition(T, k):
    partition = 0
    for energy in ham_energies(k):
        partition += np.exp(-energy/T)
    return partition

def calc_gibbs(T, k):
    H = free_ham(k)
    state = Qobj(-H/T).expm()
    return state/state.tr()

def g(T,entropy, k):
        return -T*(entropy - np.log(calc_partition(T,k)))

def find_temp(entropy, k, x0 = 1, bounds = [(10**(-5), 1)]):
    result = minimize(g, x0, args=(entropy, k), bounds=bounds)
    return result.x[0]

def calc_mean_non_eq_free_energy(state, N, k):
    mean_entropy = calc_mean_entropy(state, N, k)
    T = find_temp(mean_entropy, k)
    return calc_mean_energy(state, N, k) - T*mean_entropy + T*np.log(calc_partition(T, k))

In [ ]:
N = 8
points = 100
tfinal = 200

state = [fock(2,0)]*N
state[0] = fock(2,1)
initial_state = tensor(state)

#initial_state = random_total_state(N)
H = spin_ham(N, 0.01)

tlist = np.linspace(0,tfinal,points)
result = mesolve(H, initial_state, tlist, [])
state0 = initial_state
state1 = result.states[int(points/4)]
state2 = result.states[int(points/2)]
state3 = result.states[int(3*points/4)]
state4 = result.states[-1]
rand_state = random_total_state(N)

tot_erg0 = calc_mean_ergotropy(state0, N, N)
tot_erg1 = calc_mean_ergotropy(state1, N, N)
tot_erg2 = calc_mean_ergotropy(state2, N, N)
tot_erg3 = calc_mean_ergotropy(state3, N, N)
tot_erg4 = calc_mean_ergotropy(state4, N, N)
tot_erg_rand = calc_mean_ergotropy(rand_state, N, N)

mean_erg0, mean_erg1, mean_erg2, mean_erg3, mean_erg4, mean_erg_rand, free_energy_list = [], [], [], [], [], [], []
for k in tqdm(range(1,N+1)):
    mean_erg0.append(calc_mean_ergotropy(state0, N, k)/tot_erg0)
    mean_erg1.append(calc_mean_ergotropy(state1, N, k)/tot_erg1)
    mean_erg2.append(calc_mean_ergotropy(state2, N, k)/tot_erg2)
    mean_erg3.append(calc_mean_ergotropy(state3, N, k)/tot_erg3)
    mean_erg4.append(calc_mean_ergotropy(state4, N, k)/tot_erg4)
    mean_erg_rand.append(calc_mean_ergotropy(rand_state, N, k)/tot_erg_rand)
    #free_energy_list.append(calc_mean_free_energy(state4, N, k))
    
plt.plot(mean_erg0, label="t0")
plt.plot(mean_erg1, label="t1")
plt.plot(mean_erg2, label="t2")
plt.plot(mean_erg3, label="t3")
plt.plot(mean_erg4, label="t4")
plt.plot(mean_erg_rand, label="random")
plt.legend()
plt.show()

In [ ]:
N = 12
points = 100
tfinal = 200

state = [fock(2,0)]*N
state[0] = fock(2,1)
initial_state = tensor(state)
H = spin_ham(N, 0.01)

tlist = np.linspace(0,tfinal,points)
result = mesolve(H, initial_state, tlist, [])
final_state = result.states[-1]

mean_entropy_list = [calc_mean_entropy(final_state, N, k) for k in range(1, N+1)]
mean_energy_list = [calc_mean_energy(final_state, N, k) for k in range(1, N+1)]
mean_ergotropy_list = [calc_mean_ergotropy(final_state, N, k) for k in range(1, N+1)]
mean_non_eq_free_energy_list = [calc_mean_non_eq_free_energy(final_state, N, k) for k in range(1, N+1)]


plt.plot(mean_entropy_list)
plt.plot(mean_energy_list)
plt.plot(mean_ergotropy_list)
plt.plot(mean_non_eq_free_energy_list)
plt.show()

In [ ]:
k = 7
mean_entropy = calc_mean_entropy(final_state, N, k)
T = find_temp(mean_entropy, k)
print(entropy_vn(calc_gibbs(T, k)), mean_entropy)

In [ ]:
plt.plot([find_temp(calc_mean_entropy(final_state, N, k), k) for k in range(1,N+1)])
plt.show()

In [ ]:
k = 7
T_list = np.linspace(10**(-5), 0.5)
mean_entropy = calc_mean_entropy(final_state, N, k)

plt.plot(T_list, [-g(T,mean_entropy, k) for T in T_list])
plt.show()

In [ ]:
N = 8

state0 = random_product_state(N)
state1 = random_total_state(N)
state2 = random_total_state(N)
state3 = random_total_state(N)
state4 = random_total_state(N)


tot_erg0 = calc_mean_ergotropy(state0, N, N)
tot_erg1 = calc_mean_ergotropy(state1, N, N)
tot_erg2 = calc_mean_ergotropy(state2, N, N)
tot_erg3 = calc_mean_ergotropy(state3, N, N)
tot_erg4 = calc_mean_ergotropy(state4, N, N)

mean_erg0, mean_erg1, mean_erg2, mean_erg3, mean_erg4, mean_erg_rand = [], [], [], [], [], []
for k in tqdm(range(1,N+1)):
    mean_erg0.append(calc_mean_ergotropy(state0, N, k)/tot_erg0)
    mean_erg1.append(calc_mean_ergotropy(state1, N, k)/tot_erg1)
    mean_erg2.append(calc_mean_ergotropy(state2, N, k)/tot_erg2)
    mean_erg3.append(calc_mean_ergotropy(state3, N, k)/tot_erg3)
    mean_erg4.append(calc_mean_ergotropy(state4, N, k)/tot_erg4)
    
plt.plot(mean_erg0, label="t0")
plt.plot(mean_erg1, label="t1")
plt.plot(mean_erg2, label="t2")
plt.plot(mean_erg3, label="t3")
plt.plot(mean_erg4, label="t4")
plt.legend()
plt.show()

In [ ]:
points = 100
tfinal = 200
Nmax = 12


for k in range(2, Nmax):
    mean_erg = []
    for N in range(k, Nmax):
        state = [fock(2,0)]*N
        state[0] = fock(2,1)
        initial_state = tensor(state)
        H = spin_ham(N, 0.01)

        tlist = np.linspace(0,tfinal,points)
        result = mesolve(H, initial_state, tlist, [])
        final_state = result.states[-1]
        rand_state = random_total_state(N)
        mean_erg.append(calc_mean_ergotropy(final_state, N, k))
    plt.plot(mean_erg)
plt.show()

In [ ]:
Nmax = 12

for N in tqdm(range(2, Nmax)):
    state = [fock(2,0)]*N
    state[0] = fock(2,1)
    initial_state = tensor(state)
    H = spin_ham(N, 0.01)

    tlist = np.linspace(0,tfinal,points)
    result = mesolve(H, initial_state, tlist, [])
    final_state = result.states[-1]
    rand_state = random_total_state(N)
    tot_erg = calc_mean_ergotropy(final_state, N, N)
    
    mean_erg = []
    for k in range(1,N+1):
        mean_erg.append(calc_mean_ergotropy(final_state, N, k)/tot_erg)
        
    plt.plot(mean_erg)
plt.show()

In [ ]:
from scipy.optimize import curve_fit

def function(x, a, b, c, d):
    #return a * np.exp(b * x) 
    return a*x+b*x**2+c*x**3+d*x**4

x_data = [k for k in range(1,N+1)]
popt, pcov = curve_fit(function, x_data, mean_erg, p0=(1, 1, 1, 1))
fit =[function(x, popt[0], popt[1], popt[2], popt[3]) for x in x_data]

plt.plot(x_data, mean_erg)
plt.plot(x_data, fit)
plt.show()
popt

In [ ]:
k=30
for i in range(N):
    avg = 0
    for t in range(k):
        avg += (result.states[99-t].ptrace(i)).diag()[0]
    print(avg/k)

In [ ]:
N = 12
points = 100
tfinal = 1000

state = [fock(2,0)]*N
state[0] = fock(2,1)
initial_state = tensor(state)
H = spin_ham(N, 0.01)

tlist = np.linspace(0,tfinal,points)
result = mesolve(H, initial_state, tlist, [])

pop0 = [(result.states[t].ptrace(0)).diag()[0] for t in range(100)]
pop1 = [(result.states[t].ptrace(1)).diag()[0] for t in range(100)]
pop2 = [(result.states[t].ptrace(6)).diag()[0] for t in range(100)]
plt.plot(pop0)
plt.plot(pop1)
plt.plot(pop2)
plt.show()

In [ ]:
plt.hist(dist_ergotropy(state4, N, N-1), 50)
plt.show()

In [ ]:
ham_energies(2)